# 6章 学習のテクニック
- 学習において重要なテクニックが3つある
    1. パラメータの更新方法
        - これまでには、 x := x -λ(δL/δx)を使ってきた
        - 他に、momentum, adagrad, adamなどがある
    2. 初期パラメーターの設定
        - 0はだめ
        - 勾配消失
        - 隠れ層のアクティベーションん分布が正規化する
    3. batch normalization
        - 矯正t系にアクティベーションの分布を調整する方法
    4. 正則化
    5. ハイパーパラメータの検証
        
## パラメータの更新
- NNにおける学習の目的はLossFunctionをなるべく小さくするパラメーターを発見すること = 最適化
- これまでは、SGD: 確率的勾配降下法を使ってきたが、他にもある
- SGDの欠点は、関数の形状が等方的でない, 即ち谷のような形の場合非効率な経路で探索をしてしまうこと
- それを解消する方法として３つある
    1. Momentum
    2. AdaGrad
    3. Adam


In [ ]:
# W := W - λ(δL/δW)で更新する

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]
            

In [ ]:
- momentum
    - 新たにv: 速度を導入
        - v := αv - λ(δL/δx)
                - α: 0.9など固定値を用いる
        - W := W + v
- adagrad
    - NNでは学習係数が小さすぎると学習に時間がかかりすぎるし、大きすぎると発散してしまう
    - そこでlrを少しずつ小さくする学習係数の減衰(decay)という方法がある
    - adagradはパラメーターの要素ごとに適応的に学習係数を調整しながら学習する
        - h := h + (δL／δW) × (δL／δW) ... 要素ごとの掛け算
        - W := W - λ * h ** (-1/2) * (δL／δW)
    - 大きく更新された = よく動いたパラメーターの学習係数は次第に小さくなる
- adam
    - 詳細は論文に譲るが、効果的    

In [ ]:
## 重みの初期値
- 初期値が0, というより重みを均一にしてしまうと、誤差逆伝播法においては全ての重みが均一に更新されてしまい意味がない
- 勾配消失問題
    - 逆伝播での勾配の値が小さくなり、消える問題
    - 勾配が消失すると学習ができない(移動しなくなる)
    - sigmoidなどを利用していて、アクティベーションの出力が0,1に偏るとsigmoidの傾きがほぼゼロになり生じることがアアル
    - また、アクティベーション層の出力に偏りが無く、似たような出力をするとなると、それは複数のニューロンを保つ意味がなくなる = 表現力の制限
    - **アクティベーション層の出力は適度な広がりを持つことを求められる**
- 推奨される置き方1: Xavierの初期値
    - 各層のアクティベーション を同じ広がりのある分布にしたい
    - 前奏のノードの個数をnとした場合、n ** (-1/2)の標準偏差を持つ分布を使う
    - w = no.random.randn(node_num, node_num) / np.sqrt(node_num)
- 推奨される置き方2: Heの初期値
    - アクティベーション関数がReLU関数の場合に適した置き方
    - (2/n) ** (1/2)の標準偏差を持つ分布を使う
    

In [ ]:
## Batch Normalization
- Batch Normalizationとは
    - 強制的にアクティベーションの分布を調整する
- メリット
    1. 学習を早く進行させられる
    2. 初期値にそれほど依存しない = 神経質にならなくていい
    3. 科学集を抑制する = Dropoutの必要性を減らす
- Affine -> BatchNorm -> Activationとして、間に挟む
- ミニバッチの単位で成果を行なう
    - B = {x1, x2, ...} というm個の入力データがある
    - xiを正規化(略)したBrを用意する
    - ソレに対して固有のスケールとシフトで変換を行なう
        - yi = γxi + β ... ここでガンマとベータはパラメタ. γ=1.0, β = 0.0からスタートし、学習によって適した値に調整する？
        - [ ] 学習するパラメータをwから変えるということ？

In [ ]:
## 正則化
- 過学習の問題
    - 訓練データに特化しすぎて、汎化性能が低くなること
    - trainとtestでaccracyの向上度合いにおおきな乖離が生まれているとき、それは汎化能力が低いことを示す
- その原因
    1. パラメーターを大量に持ち、表現力の高いモデルであること
    2. 訓練データが少ないこと
- 対処1: Weight Decay(荷重減衰)
    - 学習の過程において、おおきな重みを持つことにたいしてペナルティを課す
    - 損失関数にたいして、1/2λW**2を加える => 重みwが大きいほど損失関数が大きくなる => 重みを減らそうと動く
        - [ ] L1ノルム, L2ノルム, L∞ノルムというのが出てきた。突然。
    - これは認識精度の乖離を小さくすることが目的であり、精度は当然ながら「下がる」
- 対処2: Dropout
    - ニューロンをランダムに消去しながら学習する手法
    - 隠れ層のニューロンをランダムに選び出し、そのニューロンを消去する
    - 訓練時にはデータが流れるたびにランダムに削除するニューロンを選ぶ
    - テスト時には全てのニューロンの信号を伝達するが、訓練時に消去した割合を掛けて出力をだす
    

In [ ]:
class Dropout:
    def __init__(self, drouput_ratio=0.5):
        self.dropout_ratio = 0.5
        self.mask = None
        
    def forward(self, x, train_flg=True):
        if train_flag:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
    
    def backward(self, dout):
        return dout * self.mask
    
# *x.shapeは引数で配列を展開してる

In [ ]:
## ハイパーパラメーターの検証
- 各層のニューロンの数やバッチサイズ、学習係数やweight decayなどなど
- 訓練データの中から20%ほどを検証データとして分離し、利用する
- HPの最適化においてはエポックを小さくして、1会の放火に要する時間を短縮するのが有効
- 手順
    1. 範囲を設定する ... 大体10**-3 - 10**3からまず選ぶ
    2. ランダムにサンプリング
    3. エポック小さくして学習し、認識精度評価
    4. 2-3を100回程度繰り返し、その結果からHPの良い位置に当たりをつける